# **Notebook de test de Téléchargement de donnée de basket depuis le site nba.com**

Sommaire

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, re, time #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import Connexion_Transfert as ct
from TelechargementDonnees import (JourneeSiteNba, PasDeMatchError, JoueursSiteNba, 
                                   DriverFirefox, gererCookie, simplifierNomJoueur, Blessures)
import pandas as pd
import numpy as np
import time
from collections import Counter
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import Select

## 1. Telecharger les données liees aux matchs depuis le site internet pour une ou plusieurs dates  
1. on telecharge une journee
1. on sauvegarder dans un json

In [ ]:
#telecharger une journee
try : 
    journee=JourneeSiteNba('2021-02-12', dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
    #exporter
    #journee.saveCsv()
except PasDeMatchError as e : 
    print(e)

In [ ]:
# pour un ensemble de journee : 
for j in ['2021-01-%02d' %n  for n in range(25,27)] : 
    print(j)
    try :
        journee=JourneeSiteNba(j,dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
        journee.saveCsv()
    except PasDeMatchError as e: 
        print(e)    
        continue

## 2. Telecharger les données liees aux joueurs depuis le site internet 
> Dans le cas ou on souhaite telecharger la totalite des donnees de la page des joueuers
1. on se connecte à la page principale des joueurs
1. on affiche l'integralite des donnees

In [62]:
#creation de la classe
pagejoueuer=EnsembleJoueursSiteNba()
#acces aux donnees
pagejoueuer.dfJoueurs
#export des donnees
pagejoueuer.dfJoueurs.reset_index(drop=True).to_json(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\temp\basket\Joueurs_saison_2020-2021\Joueurs_saison_2020-2021.json')

## 3. Test récupération stats_equipe
1. on balaye le site nba comme d'habitude 
1. on cree la df du match
1. on télécharge la données de match de la Bdd
1. on joint la df de match avec celle de la Bdd pour obtenir la df des stats_equipe avec l'id_match.

In [2]:
urlSiteNbaScore='https://www.nba.com/games'
nomsColonnesMatchRef=['id_equipe','q1','q2','q3','q4','final']
nomsColonnesStatsEquipe=['id_equipe','pts_in_paint','fastbreak_pts','biggest_lead',
                   'pts_banc','tm_rebonds','ball_perdu','tm_ball_perdu','pt_subi_ctrattaq']

In [3]:
with ct.ConnexionBdd('basket') as c:
    dfMatchsBdd=pd.read_sql('select id_match, date_match, equipe_domicile,equipe_exterieure   from donnees_source.match', c.sqlAlchemyConn)
    dfMatchsBdd.date_match=dfMatchsBdd.date_match.apply(lambda x : x.strftime('%Y-%m-%d'))

In [ ]:
with ct.ConnexionBdd('basket') as c:
    with DriverFirefox() as d :
        driver=d.driver
        for d in [d.strftime('%Y-%m-%d') for d in pd.date_range(start='15/01/2021',periods=1)] : 
            urlDateJournee=fr'{urlSiteNbaScore}?date={d}'
            driver.get(urlDateJournee)
            time.sleep(3)
            gererCookie(driver)
            #recuperer la liste des hyperliens qui ont le mot "feuille" dedans
            try :
                elementsScore=WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.LINK_TEXT, 'BOX SCORE')))
            except TimeoutException as e :
                print(e)
                raise PasDeMatchError(d)
            listBoxScore=[p.get_attribute("href") for p in elementsScore]
            dicoJournee={}
            for e,p in enumerate(listBoxScore) : 
                print(d,e,p)
                driver.get(p)
                time.sleep(7)
                driver.implicitly_wait(20)
                #bascule sur l'onglet summary er recuperer les stats de matchs: 
                elementSummary=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT, 'Summary')))
                driver.execute_script("arguments[0].click();", elementSummary)#passer via Javascript pour clicker mm si qqch devant
                time.sleep(7)
                driver.implicitly_wait(20)
                dicoJournee[e]={'match':pd.read_html(driver.page_source)[0]}
                dicoJournee[e]['stats_equipes']=pd.concat([dicoJournee[e]['match'][['Unnamed: 0']],
                                                          pd.read_html(driver.page_source)[1]],axis=1)
            for i in range(len(dicoJournee)) :
                nbColonnesMatch=len(dicoJournee[i]['match'].columns) - len(nomsColonnesMatchRef)
                if nbColonnesMatch > 0 : 
                    prolongation=[f'pr{i+1}' for i in range(nbColonnesMatch)]
                    nomsColonnesMatch=nomsColonnesMatchRef[:-1]+prolongation+[nomsColonnesMatchRef[-1]]
                else : 
                    nomsColonnesMatch=nomsColonnesMatchRef
                dicoJournee[i]['match'].columns=nomsColonnesMatch
                dicoJournee[i]['stats_equipes'].columns=nomsColonnesStatsEquipe
                equipeExt, equipeDom=dicoJournee[i]['match'].iloc[0].id_equipe, dicoJournee[i]['match'].iloc[1].id_equipe
                dicoJournee[i]['dfMatch']=pd.DataFrame({'date_match':[d,],'equipe_domicile':[equipeDom],'equipe_exterieure':[equipeExt]})
                dicoJournee[i]['stats_equipes']['id_match']=dicoJournee[i]['dfMatch'].merge(dfMatchsBdd, on=['date_match','equipe_domicile','equipe_exterieure']).id_match.values[0]   
                dicoJournee[i]['stats_equipes'].to_sql('stats_equipes',c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)